## Exploration base de données

In [1]:
import pandas as pd

In [2]:
pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-05-2020.csv', sep=',')

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,Mainland China,2020-03-05T14:53:03,67466,2902,40592,30.9756,112.2707
1,NaN,South Korea,2020-03-05T09:03:09,6088,35,41,36.0000,128.0000
2,NaN,Italy,2020-03-05T17:43:03,3858,148,414,43.0000,12.0000
3,NaN,Iran,2020-03-05T13:43:04,3513,107,739,32.0000,53.0000
4,Guangdong,Mainland China,2020-03-05T09:23:03,1351,7,1181,23.3417,113.4244
...,...,...,...,...,...,...,...,...
168,NaN,Ukraine,2020-03-03T15:33:02,1,0,0,48.3794,31.1656
169,From Diamond Princess,Australia,2020-02-29T02:03:10,0,0,0,35.4437,139.6380
170,"Lackland, TX (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,29.3829,-98.6134
171,"Omaha, NE (From Diamond Princess)",US,2020-02-24T23:33:02,0,0,0,41.2545,-95.9758
